# Accessibility API

Prototyping of a wrap of accessibility computation into custom classes with simple API.

In [1]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import joblib

from indicators import Accessibility

/var/folders/2f/fhks6w_d0k556plcv3rfmshw0000gn/T/ipykernel_63819/1239877917.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
data_folder = "/Users/martin/Library/CloudStorage/OneDrive-SharedLibraries-TheAlanTuringInstitute/Daniel Arribas-Bel - demoland_data"

Load the travel time matrix.

In [3]:
ttm = pd.read_parquet(f"{data_folder}/processed/accessibility/ttm_complete.parquet")

Convert the matrix to boolean xarray.DataArray.

In [4]:
ttm = ttm.set_index(["from_id", "to_id"])
ttm.columns = ["transit", "car", "bike", "walk"]
ttm.columns.name = "mode"
ttm_arr = xr.DataArray.from_series(ttm.stack())
ttm_15 = ttm_arr <= 15
ttm_15.name = "ttm_15"

Load Workplace zone population.

In [5]:
wpz_population = pd.read_csv(
    f"{data_folder}/processed/accessibility/wpz_tynewear_occupation_edited.csv"
).rename(columns={"wpz11cd": "to_id"}).set_index("to_id")["pop"]

Merge with the traveltime matrix to create a baseline.

In [6]:
da = xr.DataArray.from_series(wpz_population)
da.name = "wpz_population"
baseline = xr.merge([ttm_15, da])
baseline["wpz_population"] = baseline["wpz_population"].fillna(0)

In [7]:
baseline

<xarray.Dataset>
Dimensions:         (from_id: 3795, to_id: 9254, mode: 4)
Coordinates:
  * from_id         (from_id) object 'E00041363' 'E00041364' ... 'E00175605'
  * to_id           (to_id) object 'E00041363' ... 'idFFE0D1A6-2B10-40AE-8E6E...
  * mode            (mode) object 'transit' 'car' 'bike' 'walk'
Data variables:
    ttm_15          (from_id, to_id, mode) bool True True True ... False False
    wpz_population  (to_id) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

Load greenspace data.

In [8]:
acc_greenspace = pd.read_csv(
    f"{data_folder}/processed/accessibility/acc_greenspace_allmodes_15min_tynewear.csv",
    index_col=0
)

In [9]:
acc_greenspace.columns = ["transit", "car", "bike", "walk"]

In [10]:
greenspace = xr.DataArray.from_series(acc_greenspace.stack()).rename({"level_1":"mode"})
greenspace.name = "green_accessibility"

In [11]:
baseline = xr.merge([baseline, greenspace])
baseline["green_accessibility"] = baseline["green_accessibility"].fillna(0)

Create a wrapper class.

In [12]:
acc = Accessibility(baseline)

Test the object on example input.

Create random OA data.

In [13]:
oa_data = pd.Series(np.random.randint(-100, 100, len(baseline.from_id)), index=baseline.from_id.values, name="oa")
oa_data.index.name = "to_id"

In [14]:
oa_data

to_id
E00041363    38
E00041364    78
E00041366   -56
E00041367    76
E00041368    48
             ..
E00175601    20
E00175602   -16
E00175603    32
E00175604   -81
E00175605    51
Name: oa, Length: 3795, dtype: int64

Compute accessibility.

In [15]:
acc.job_accessibility(oa_data, "walk")

<xarray.DataArray 'combined' (from_id: 3795)>
array([9441., 9451., 2558., ...,  675., 1237.,  401.])
Coordinates:
    mode     <U4 'walk'
  * from_id  (from_id) object 'E00041363' 'E00041364' ... 'E00175605'

In [17]:
new_green = pd.Series(np.random.randint(-10000, 10000, len(baseline.from_id)), index=baseline.from_id.values, name="oa")
new_green.index.name = "to_id"

In [18]:
acc.greenspace_accessibility(new_green, "walk")

<xarray.DataArray (from_id: 3795)>
array([ 525753.89095001,  562085.30120002,  260592.45855   , ...,
       1861925.73930004, 1760153.35770002,   66662.1107    ])
Coordinates:
    mode     <U4 'walk'
  * from_id  (from_id) object 'E00041363' 'E00041364' ... 'E00175605'

Save the custom class to a compressed joblib.

In [19]:
with open(f"{data_folder}/models/accessibility.joblib", "wb") as f:
    joblib.dump(acc, f, compress=True)

Test loaded class

In [3]:
with open(f"{data_folder}/models/accessibility.joblib", "rb") as f:
    acc2 = joblib.load(f)

In [4]:
acc2

In [7]:
oa_data = pd.Series(np.random.randint(-100, 100, len(acc2.baseline.from_id)), index=acc2.baseline.from_id.values, name="oa")
oa_data.index.name = "to_id"
acc2.job_accessibility(oa_data, "walk")

<xarray.DataArray 'combined' (from_id: 3795)>
array([ 9367., 10412.,  2439., ...,   967.,  1891.,   504.])
Coordinates:
    mode     <U4 'walk'
  * from_id  (from_id) object 'E00041363' 'E00041364' ... 'E00175605'

In [8]:
new_green = pd.Series(np.random.randint(-10000, 10000, len(acc2.baseline.from_id)), index=acc2.baseline.from_id.values, name="oa")
new_green.index.name = "to_id"
acc2.greenspace_accessibility(new_green, "walk")

<xarray.DataArray (from_id: 3795)>
array([ 407055.89095001,  438732.30120002,  298602.45855   , ...,
       1777473.73930004, 1744018.35770002,   -9878.8893    ])
Coordinates:
    mode     <U4 'walk'
  * from_id  (from_id) object 'E00041363' 'E00041364' ... 'E00175605'